# <span style="color:green">Предобработка данных</span>

### Используемый источник:
- [Первичный анализ данных](https://habr.com/ru/company/ods/blog/322626/) 


# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Основная задача проекта** — выполнить предобработку данных (подготовить к дальнейшему анализу), сделать первичные выводы. 

### Краткий план действий

1. **[Открыть файл](#section_id) с данными data.csv и изучить общую информацию data.csv**


2. **[Предобработка данных](#section_id_2)** 
     1. Определить и заполнить пропущенные значения
     1. Заменить вещественный тип данных на целочисленный
     1. Удалить дубликаты
     1. Выделить леммы в значениях столбца
     1. Категоризировать данные   


3. **[Выводы](#section_id_3)**

### Описание данных

#### clients_data (статистика о платёжеспособности клиентов)


- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

<a id='section_id'></a>
### Шаг 1. Откроем файл с данными и изучим общую информацию. 

In [1]:
# Импортируем необходимые для проекта библиотеки.
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

In [2]:
# Импортируем данные, выведим на экран первичную информацию и таблицу.
data = pd.read_csv('data.csv')
clients_data = data.copy()
display(clients_data.info(), clients_data.describe().round(), clients_data.head().round())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


None

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.0,19351.0,21525.0,21525.0,21525.0,21525.0,19351.0
mean,1.0,63046.0,43.0,1.0,1.0,0.0,167422.0
std,1.0,140827.0,13.0,1.0,1.0,0.0,102972.0
min,-1.0,-18389.0,0.0,0.0,0.0,0.0,20667.0
25%,0.0,-2747.0,33.0,1.0,0.0,0.0,103053.0
50%,0.0,-1203.0,42.0,1.0,0.0,0.0,145018.0
75%,1.0,-291.0,53.0,1.0,1.0,0.0,203435.0
max,20.0,401755.0,75.0,4.0,4.0,1.0,2265604.0


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8438.0,42,высшее,0,женат / замужем,0,F,сотрудник,0,253876.0,покупка жилья
1,1,-4025.0,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.0,приобретение автомобиля
2,0,-5623.0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145886.0,покупка жилья
3,3,-4125.0,32,среднее,1,женат / замужем,0,M,сотрудник,0,267629.0,дополнительное образование
4,0,340266.0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.0,сыграть свадьбу


In [3]:
# Проверим строки таблиц на наличие дубликатов.
print(f'Количество дубликатов состовляет: {clients_data.duplicated().sum()}')

Количество дубликатов состовляет: 54


In [4]:
# Посмотри на уникальные значения 'dob_years'.
clients_data['dob_years'].sort_values().unique()

array([ 0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75], dtype=int64)

0 это ничего, нужно исправить на среднюю.

In [5]:
# Посмотрим уникальные значения по столбцам 'education', 'family_status', 'gender', 'income_type', 'purpose'.
for column in ['education', 'family_status', 'gender', 'income_type', 'purpose']:
    print(f'Уникальные слова {column}: {clients_data[column].unique()}')
    print()

Уникальные слова education: ['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']

Уникальные слова family_status: ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']

Уникальные слова gender: ['F' 'M' 'XNA']

Уникальные слова income_type: ['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']

Уникальные слова purpose: ['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'оп

### <span style="color:red"> Замечания по данным</span>
1. **'children'** — В столбце есть отрицательные значения, необходимо исправить и число 20 смущает, скорее всего опечатка, исправим на 2, нолик лишний.
2. **'days_employed'** — наличие пропусков, тип данных стоит изменить на целочисленный (int).  Так же в значениях присутствуют отрицательные числа. Наличие слишком больших значений.
3. **'dob_years'** — отрицательные значения. Также возраст клиента не может быть 0, стоит исправить.
4. **'education'** — в столбце разный регистр символов.
5. **'gender'** — непонятный третий пол клиента, стоит разобраться.
6. **'total_income'** — пропуски, черезчур высокие значения по ежемесячному доходу. Стоит изменить тип данных на целочисленные.
7. **'purpose'** — очень много словосочетаний схожих по смыслу.
8. В таблице найдены 54 дубликата, их стоит удалить.

<a id='section_id_2'></a>
### Шаг 2. Предобработка данных

In [6]:
# Устраним замечания по столбцу children.
clients_data['children'] = clients_data['children'].abs() # в столбце children убрал минусы.
clients_data['children'] = clients_data['children'].replace(20, 2) # заменил 20 на 2.


Чтобы обработать слишком большие значения, предположим, что некоторые числа отражают дни, а некоторые часы. 
Предположим, что клиент 18 лет до 75 лет (75 это max по нашей таблице данных), смог бы отработать не больше 15000 дней, соответственно, все числа что больше этого значения, принемаем как за часы. 

In [7]:
# Устраним замечания по столбцу days_employed.
clients_data['days_employed'] = clients_data['days_employed'].abs().fillna(0) # убрали минусы, пропуски заменили нолями.
mean_days_employed = clients_data['days_employed'].mean() # медиана days_employed.
clients_data['days_employed'] = clients_data['days_employed'].replace(0, mean_days_employed ) # заменили (0) на медиану.

list_days_employed = []

# Приведем часы трудового стажа в дни.
for value in clients_data['days_employed']:
    if value >= 15000:
        list_days_employed.append(value//24)
    else:
        list_days_employed.append(value)
        
# Создадим новый столбец 'new_days_employed' на основе списка list_days_employed.
clients_data['new_days_employed'] = list_days_employed
# и удалим страый
clients_data = clients_data.drop(['days_employed'], axis=1) 

In [8]:
# По столбцу dob_years заменим (0) на среднюю.
mean_age = clients_data['dob_years'].sum() // len(clients_data[clients_data['dob_years'] !=0]) # средний возраст = 43.
clients_data['dob_years'] = clients_data['dob_years'].replace(0, mean_age) # Заменим (0) на средний возраст.

In [9]:
# Приведем солбец education к одному регистру букв.
clients_data['education'] = clients_data['education'].str.lower()

In [10]:
# Заменим пропуски по столбцу total_income на среднее значение.
mean_total_income = clients_data['total_income'].sum() / (len(clients_data['total_income']) - 2174)   
clients_data['total_income'] = clients_data['total_income'].fillna(value= mean_total_income)

1. Лишний ноль в столбце children - возможно опечатка
2. Отрицательные значения, пропуски в виде NaN - возможно проставились при формировании списка из разных таблиц, от сюда и разные зачения по столбцу days_employed (в часах, в днях).

### Изменение типа данных

In [11]:
# Изменим тип данных в столбцах days_employed и total_income float на int.
clients_data['new_days_employed'] = clients_data['new_days_employed'].astype('int')
clients_data['total_income'] = clients_data['total_income'].astype('int')

### Обработка дубликатов

In [12]:
# Удалим дубликаты из нашей таблицы.
clients_data = clients_data.drop_duplicates().reset_index(drop=True)

Методом drop_duplicates() удалили все строки дубли в таблице clients_data


### Лемматизация

In [13]:
# Поскольку в нашем столбце 'purpose' много различных слов схожих по теме, 
# нужно определить основные темы и распределить словосочетания по ним.
m = Mystem()
# Создадим список уникальных значений столбца 'purpose'.
purpose = clients_data['purpose'].unique()
lemmas_purpose = []
for element in purpose:
    lemmas = m.lemmatize(element)
    lemmas_purpose.append(lemmas)

In [14]:
# При помощи списка генератора удалим все слова символы длина которых меньше трех.
for list_lem in lemmas_purpose:   
    list_lem[:] = [x for x in list_lem if len(x) > 2]
# Избавимся от вложенности списков.  
lemmas_purpose = [lem for list_lem in lemmas_purpose for lem in list_lem]
    
print(lemmas_purpose)

['покупка', 'жилье', 'приобретение', 'автомобиль', 'дополнительный', 'образование', 'сыграть', 'свадьба', 'операция', 'жилье', 'образование', 'проведение', 'свадьба', 'покупка', 'жилье', 'для', 'семья', 'покупка', 'недвижимость', 'покупка', 'коммерческий', 'недвижимость', 'покупка', 'жилой', 'недвижимость', 'строительство', 'собственный', 'недвижимость', 'недвижимость', 'строительство', 'недвижимость', 'покупка', 'подержать', 'автомобиль', 'покупка', 'свой', 'автомобиль', 'операция', 'коммерческий', 'недвижимость', 'строительство', 'жилой', 'недвижимость', 'жилье', 'операция', 'свой', 'недвижимость', 'автомобиль', 'заниматься', 'образование', 'сделка', 'подержанный', 'автомобиль', 'получение', 'образование', 'автомобиль', 'свадьба', 'получение', 'дополнительный', 'образование', 'покупка', 'свой', 'жилье', 'операция', 'недвижимость', 'получение', 'высокий', 'образование', 'свой', 'автомобиль', 'сделка', 'автомобиль', 'профильный', 'образование', 'высокий', 'образование', 'покупка', 'жил

In [15]:
# Посчитаем количество совпадений уникальных лем.
counter_lemmas_purpose = Counter(lemmas_purpose)
print(counter_lemmas_purpose)

Counter({'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'операция': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'подержать': 1, 'подержанный': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1})


In [16]:
# По сформированному словарю видно какое слово повторялось чаще, 
#можно из них составить список нужных нам обопщающих слов.
list_subject = ['недвижимость', 'автомобиль', 'образование', 'жилье', 'свадьба']

Как видим у нас получилось всего 5 основных целей. Жилье и недвижимость можно объединить. Изменим столбец 'purpose', распределив все значения по нашим четырем целям.

In [17]:
# Создадим функцию, которая проверяет в словосочетаниях наличие отрывков лем и если такие есть
# возвращает нужное слово.
def lem(series):
    if 'недвижим' in series or 'жил' in series:
        return 'недвижимость'
    elif 'автомоби' in series:
        return 'автомобиль'
    elif 'образован' in series:
        return 'образование'
    elif 'свадьб' in series:
        return 'свадьба'
    else:
        return "Неопределено"
        
clients_data['purpose'] = clients_data['purpose'].apply(lem)

clients_data.head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,new_days_employed
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,8437
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,4024
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,5623
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,4124
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,14177


In [18]:
# Проверим, есть ли нераспределенные слова.
clients_data['purpose'].unique()

array(['недвижимость', 'автомобиль', 'образование', 'свадьба'],
      dtype=object)

Как видим, нераспределенных слов нет, все словосочетания распределились по споим группам.

In [19]:
# Далее мы подобным образом, только уже без лем, распределим клиентов по группам относительно их ежемесячного дохода.
def income_group(income):
    if  income <= 30000:
        return 'зарплата низкая'
    if 30000 < income <= 100000:
        return 'зарплата средняя'
    return 'зарплата высокая'
clients_data['income_group'] = clients_data['total_income'].apply(income_group)

# Теперь таблица готова к дальнейшему ее анализу.
clients_data.head()

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,new_days_employed,income_group
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,8437,зарплата высокая
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,4024,зарплата высокая
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,5623,зарплата высокая
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,4124,зарплата высокая
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,14177,зарплата высокая


### Категоризация данных

In [20]:
# Создадим функцию 'group_data', которая будет формировать нам таблицы 
# сгруппированных по определенным категориям и показывать долю возврата кредитов в срок.
def group_data(data, index, column, values):
    new_data = data.pivot_table(index=index,\
                                  columns=column,\
                                  values=values,\
                                  aggfunc= 'count').reset_index()
    # Переименуем колонки.
    new_data.columns = [index,'not_debt', 'debt']
    # Для наглядности добавим столбец общее количество клиентов 'sum_clients', по группам.
    new_data['sum_clients'] = new_data['not_debt'] + new_data['debt']
    # Посмотрим на долю возврата.
    new_data['return_share_%'] = ((new_data['not_debt'] / new_data['sum_clients']) * 100).round()
    return new_data  

In [21]:
# Проверим: Есть ли зависимость между наличием детей и возвратом кредита в срок.
# Для этого сгруппируем данные в таблицу 'group_children' по столбцу 'children' и посчитаем количество строк 
# для каждой группы (1-долг, 0 - отсутствие долга).
group_data(clients_data, 'children', 'debt', 'education_id')

,children,not_debt,debt,sum_clients,return_share_%
0,0,13028.0,1063.0,14091.0,92.0
1,1,4410.0,445.0,4855.0,91.0
2,2,1926.0,202.0,2128.0,91.0
3,3,303.0,27.0,330.0,92.0
4,4,37.0,4.0,41.0,90.0
5,5,9.0,NaN,NaN,NaN


Из таблицы видно, что зависимости между наличием детей и возвратом кредита в срок нет, а если и есть то несущественная.

In [22]:
# Проверим: Есть ли зависимость между уровнем дохода и возвратом кредита в срок.
# Для этого сгруппируем данные в таблицу 'group_family_status' по столбцу 'family_status' и посчитаем количество строк 
# для каждой группы.
group_data(clients_data, 'family_status', 'debt', 'family_status_id')

,family_status,not_debt,debt,sum_clients,return_share_%
0,Не женат / не замужем,2536,274,2810,90.0
1,в разводе,1110,85,1195,93.0
2,вдовец / вдова,896,63,959,93.0
3,гражданский брак,3763,388,4151,91.0
4,женат / замужем,11408,931,12339,92.0


Максимальное различие возвратом кредита в срок мы видим в 3%.Так же несущественное различие, чтобы делать какие-нибудь выводы. Хотя учесть данные стоит. Все таки у группы Не женат / не замужем доля возврата наименьшая.

In [23]:
# Проверим: Есть ли зависимость между семейным положением и возвратом кредита в срок.
# Для этого сгруппируем данные в таблицу 'group_income' по столбцу 'income_group' и посчитаем количество строк 
# для каждой группы.
group_data(clients_data, 'income_group', 'debt', 'family_status_id')

,income_group,not_debt,debt,sum_clients,return_share_%
0,зарплата высокая,15604,1387,16991,92.0
1,зарплата низкая,20,2,22,91.0
2,зарплата средняя,4089,352,4441,92.0


Странно, но цыфры говорят нам, что по заданным критериям существенных различий нет. 

<a id='section_id_3'></a>
### Шаг 4. Вывод

Изучив предоставленные нам данные, мы выявили и устранили множество пропусков, дубликатов и других неверных значений. Распределили клиентов по различным группам, проверили зависимости груп к возвратам кредита в срок. Первичный анализ данных нам показали, что существенных различий между группами и возвратам кредита в срок нет. 